<a href="https://colab.research.google.com/github/Novadotgg/AQI/blob/main/Crop_and_soil_recommendation_with_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok
!pip install flask_session
!pip install pyngrok
from flask import Flask, request, jsonify, session, render_template, render_template_string
from flask_ngrok import run_with_ngrok
import joblib
import nltk
from nltk.tokenize import word_tokenize
import spacy
from flask_session import Session
!pip install flask-ngrok
!pip install flask-session
!pip install joblib
!pip install spacy
!python -m spacy download en_core_web_sm
nltk.download('punkt')
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet
import seaborn as sns
from sklearn.model_selection import  train_test_split
import requests
from flask import Flask, request, jsonify, render_template_string, session
import joblib
from flask_session import Session
from pyngrok import ngrok
!pip install transformers
import numpy as np
from transformers import pipeline
import re
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from datetime import date
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import csv
import json
import requests
from datetime import datetime, timedelta
!ngrok authtoken 2jB3oS2xptgkYHLMldb5uNwjHEF_3k3Q4UuBthT1xD6nyitXA
crop_model = joblib.load('/content/adaboost_model_soil.pkl')
intent=joblib.load('/content/intent_model.pkl')
vectorizer=joblib.load('/content/vectorizer_intent.pkl')
soil_model=joblib.load('/content/soilpred.pkl')
forecast_model=joblib.load('/content/prophet.pkl')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
intent_classifier = pipeline("text-classification", model="distilbert-base-uncased")
sentiment_analyzer = pipeline("sentiment-analysis")
response_generator = pipeline("text-generation", model="gpt2")

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Initialize session management
app.config['SESSION_TYPE'] = 'filesystem'
Session(app)

def fetch_weather_data(location):
    api_key = '3b0880537a825f35ec5cc49574db842e'
    base_url = f'http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric'
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        rainfall = data.get('rain', {}).get('1h', 0)
        return temperature, humidity, rainfall
    else:
        return None, None, None

questions = [
    'Please provide the Nitrogen (N) value:',
    'Please provide the Phosphorous (P) value:',
    'Please provide the Potassium (K) value:',
    'Please provide the pH value:'
]

# NLP functions
def detect_intent(user_message):
    result = intent_classifier(user_message)
    intent = result[0]['label']
    confidence = result[0]['score']
    return intent, confidence

def analyze_sentiment(user_message):
    result = sentiment_analyzer(user_message)
    sentiment = result[0]['label']
    return sentiment

def generate_response(context):
    response = response_generator(context, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

def extract_parameters(user_message):
    values = {}
    match = re.search(r'n\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['N'] = int(match.group(1))
    match = re.search(r'p\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['P'] = int(match.group(1))
    match = re.search(r'k\s*=\s*(\d+)', user_message, re.IGNORECASE)
    if match:
        values['K'] = int(match.group(1))
    match = re.search(r'ph\s*=\s*(\d+\.?\d*)', user_message, re.IGNORECASE)
    if match:
        values['pH'] = float(match.group(1))
    return values

@app.route('/')
def index():
    return render_template_string('''<!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Crop Recommendation Chatbot</title>
        <style>
            body, html { height: 100%; margin: 0; display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; }
            #chat-container { display: flex; flex-direction: column; align-items: center; width: 100%; max-width: 600px; }
            #chat-box { width: 100%; height: 300px; border: 1px solid #ccc; padding: 10px; margin-bottom: 10px; overflow-y: auto; text-align: left; }
            #input-container { display: flex; width: 100%; margin-bottom: 10px; }
            #user-input { flex: 1; padding: 10px; border: 1px solid #ccc; border-radius: 100px; }
            button { padding: 10px 20px; margin-left: 10px; border-radius: 100px; cursor: pointer; border: none; background-color: #006400; color: white; }
            button:hover { background-color: #45a049; color: white; border-radius: 100px; }
            h1 { margin-bottom: 20px; text-align: center; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.2); font-family: 'Times New Roman', Times, serif; }
        </style>
    </head>
    <body>
        <div id="chat-container">
            <h1>Crop Recommendation Chatbot</h1>
            <div id="chat-box"></div>
            <div id="input-container">
                <input type="text" id="user-input" placeholder="Type your message...">
                <button onclick="sendMessage()">Send</button>
            </div>
        </div>
        <script>
            async function sendMessage() {
                const userInput = document.getElementById('user-input').value;
                const chatBox = document.getElementById('chat-box');
                chatBox.innerHTML += `<p>User: ${userInput}</p>`;

                let response = await fetch('/chatbot', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ message: userInput })
                });
                response = await response.json();
                chatBox.innerHTML += `<p>Cropiee: ${response.reply}</p>`;
                chatBox.scrollTop = chatBox.scrollHeight;
                document.getElementById('user-input').value = '';
            }

            document.getElementById('user-input').addEventListener('keydown', (event) => {
                if (event.key === 'Enter') {
                    event.preventDefault();
                    sendMessage();
                }
            });
        </script>
    </body>
    </html>
    ''')

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def classify_intent(user_message):
    user_message_vector = vectorizer.transform([user_message])
    predicted_intent = intent.predict(user_message_vector)
    return predicted_intent[0]

@app.route('/chatb', methods=['POST'])
def chatb():
    greet = ['hi', 'hello', 'wassup', 'heya', 'hey']
    user_message = request.json['message'].lower()
    if user_message in greet:
      return jsonify({'reply': "Hello! 😊 You can ask me for:  <br>🌿Crop recommendation <br>🌱Soil elements advice!!"})


    # Determine Intent
    intent = classify_intent(user_message)
    if intent == 'Crop Recommend':
        return recommend_crop(user_message)
    elif intent == 'Soil Character':
        return handle_soil_conditions(user_message)
    return jsonify({'reply': "I'm not sure how to help with that. Could you rephrase?"})

# Crop recommendation function
def recommend_crop(user_message):
    recommend_synonyms = ['recommend', 'suggest', 'advise', 'what crops can i grow', 'what should i plant', 'crop recommendation', 'yes', 'ya']
    positive_feedback = ['thanks', 'great', 'good', 'helpful', 'useful', 'love', 'like', 'happy', 'nic']
    negative_feedback = ['bad', 'not good', 'useless', 'hate', 'dislike', 'terrible', 'worst']
    crop_durations = {
        "bajra": 90, "barley": 120, "turmeric": 250, "tur": 180, "sugarcane": 365,
        "soybeans": 100, "ragi": 120, "potato": 90, "onion": 120, "maize": 100,
        "ladyfinger": 60, "jute": 150, "jowar": 120, "green gram": 70, "cotton": 180,
        "coffee": 1200, "chickpea": 110, "cabbage": 90, "wheat": 150, "rice": 40,
        "Paddy(Dhan)(Common)": 120
    }

    # Start recommendation
    if any(word in user_message.lower() for word in recommend_synonyms):
        session['intent'] = 'recommend_crop'
        session['step'] = 0
        session['data'] = []
        session['recommendations'] = []
        session['feedback_stage'] = False
        session['state'] = None
        return jsonify({'reply': '🏠Please provide your district:'})

    # Feedback handling
    if session.get('feedback_stage', False):
        sentiment = analyze_sentiment(user_message)
        if any(word in user_message.lower() for word in negative_feedback) or sentiment == 'NEGATIVE':
            session['feedback_stage'] = False
            return jsonify({'reply': 'I understand your concerns. For now, ask if you need any recommendation.'})
        elif any(word in user_message.lower() for word in positive_feedback) or sentiment == 'POSITIVE':
            session['feedback_stage'] = False
            return jsonify({'reply': 'I\'m glad to hear that! <br><br>Would you like to get some more recommendations?'})
        else:
            return jsonify({'reply': "Thank you for your feedback. If you have more to share, please let me know."})

    # Yes/No responses
    if user_message.lower() in ['yes', 'ya'] and session.get('intent') == 'recommend_crop':
        session['intent'] = 'recommend_crop'
        session['step'] = 0
        session['data'] = []
        session['recommendations'] = []
        session['feedback_stage'] = False
        return jsonify({'reply': 'Please provide your district:'})
    elif user_message.lower() in ['no', 'nah'] and session.get('intent') == 'recommend_crop':
        return jsonify({'reply': 'Thanks for your time!'})

    # Step 0: Get location and fetch weather
    if session.get('intent') == 'recommend_crop' and session['step'] == 0:
        session['location'] = user_message
        temperature, humidity, rainfall = fetch_weather_data(session['location'])

        if None in (temperature, humidity, rainfall):
            return jsonify({'reply': 'Could not fetch weather data for the provided location. Please try again with a valid location.'})

        session['weather'] = {'temperature': temperature, 'humidity': humidity, 'rainfall': rainfall}
        session['step'] += 1
        return jsonify({'reply': '🏡Which state do you belong to? '})

    # Step 1: Get state
    if session.get('intent') == 'recommend_crop' and session['step'] == 1:
        session['state'] = user_message.title()
        session['step'] += 1
        return jsonify({'reply': questions[0]})

    # Step 2+: Collect input data
    if session.get('intent') == 'recommend_crop':
        step = session['step'] - 2
        try:
            session['data'].append(float(user_message))
        except ValueError:
            return jsonify({'reply': "Please enter a valid data."})

        if step < len(questions) - 1:
            session['step'] += 1
            return jsonify({'reply': questions[session['step'] - 2]})
        else:
            try:
                features = session['data'] + [
                    session['weather']['temperature'],
                    session['weather']['humidity'],
                    session['weather']['rainfall']
                ]
                features = np.array(features, dtype=float).reshape(1, -1)
                predicted_crops = crop_model.predict(features)
                session['recommendations'] = predicted_crops

                if not session['recommendations']:
                    return jsonify({'reply': 'No suitable crops found based on your input.'})

                response_text = "🍃Based on the provided data: <br><br>"


#------------------------------------------------------------------------------------------------------------------------
                if isinstance(session['recommendations'], np.ndarray):
                    crop = session['recommendations'][0]
                else:
                    crop = session['recommendations']

                crop = str(crop).title()
                state = str(session['state']).title()
                district = str(session['location']).title()
                crop_duration=crop_durations.get(crop,120)


                url = "https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24"
                api_key = "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b"
                end_date = datetime.today()
                start_date = end_date - timedelta(days=30)
                # start_date = end_date - timedelta(days=crop_duration*4)

                all_data = []

                for single_date in pd.date_range(start_date, end_date):
                    date_str = single_date.strftime("%d-%m-%Y")

                    params = {
                        "api-key": api_key,
                        "format": "json",
                        "limit": "500",
                        "filters[Arrival_Date]": date_str,
                        "filters[Commodity]": crop,      # Properly formatted crop
                        "filters[District]": district,   # Properly formatted district
                        "filters[State]": state,         # Properly formatted state
                    }

                    try:
                        response = requests.get(url, params=params)
                        print("[DEBUG] Request URL:", response.url)  # Check the exact URL
                        data = response.json()
                        records = data.get("records", [])

                        if records:
                            all_data.extend(records)
                    except requests.exceptions.RequestException as e:
                        print(f"Error fetching data for {date_str}: {str(e)}")

                if all_data:
                    df = pd.DataFrame(all_data)
                    df.to_csv("mat.csv", index=False)
                    df=pd.read_csv("/content/mat.csv")
                    df['State'] = df['State'].str.lower()
                    df['Commodity'] = df['Commodity'].str.lower()
                    df['District']=df['District'].str.lower()
                    df['Market']=df['Market'].str.lower()
                    df['Commodity']=df['Commodity'].str.lower()
                    df['Variety']=df['Variety'].str.lower()
                    df['Grade']=df['Grade'].str.lower()
                    # print(df['District'])
                    # print(df['State'])
                    df_encoded = df.copy()
                    categorical_cols = ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Arrival_Date', 'Commodity_Code']
                    df_encoded[categorical_cols] = df_encoded[categorical_cols].astype('category').apply(lambda x: x.cat.codes)
                    corr_matrix = df_encoded.corr()
                    categorical_cols = ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Arrival_Date', 'Commodity_Code']
                    df_encoded[categorical_cols] = df_encoded[categorical_cols].astype('category').apply(lambda x: x.cat.codes)
                    df['Arrival_Date'] = pd.to_datetime(df['Arrival_Date'], format='%d/%m/%Y', errors='coerce')
                    df.dropna(subset=['Arrival_Date'], inplace=True)
                    df = df.rename(columns={'Arrival_Date': 'ds', 'Modal_Price': 'y'})

                    from prophet import Prophet
                    model = Prophet(
                        changepoint_prior_scale=0.05,
                        seasonality_mode='additive',
                        yearly_seasonality=True,
                        weekly_seasonality=True,
                        daily_seasonality=True,
                        growth='linear'
                    )
                    model.add_seasonality(name='monthly', period=30, fourier_order=10)
                    # model.add_seasonality(name='yearly', period=30, fourier_order=10)
                    df_new=df[['ds','y']].copy()
                    df_new.dropna(subset=['ds', 'y'], inplace=True)
                    model.fit(df_new)

                    future = model.make_future_dataframe(periods=crop_duration)
                    # future = model.make_future_dataframe(periods=180)
                    forecast = model.predict(future)

                    today = datetime.today()
                    current_month = today.month
                    current_year = today.year
                    next_month = (today + pd.DateOffset(months=crop_duration/30)).month
                    next_year = (today + pd.DateOffset(months=crop_duration/30)).year

                    current_month_data = forecast[(forecast['ds'].dt.year == current_year) &
                                                  (forecast['ds'].dt.month == current_month)].copy()
                    next_month_data = forecast[(forecast['ds'].dt.year == next_year) &
                                                (forecast['ds'].dt.month == next_month)].copy()

                    if current_month_data.empty or next_month_data.empty:
                        profit_loss_msg = f"{crop.capitalize()}: No forecast data available.<br><br>"
                    else:
                        current_month_data['day'] = current_month_data['ds'].dt.day
                        next_month_data['day'] = next_month_data['ds'].dt.day
                        merged_data = pd.merge(current_month_data, next_month_data, on='day', suffixes=('_current', '_next'))
                        ###
                        merged_data['price_diff'] = merged_data['yhat_next'] - merged_data['yhat_current']
                        positive_count = (merged_data['price_diff'] > 0).sum()
                        negative_count = (merged_data['price_diff'] < 0).sum()
                        #
                        # print(positive_count)
                        # print(negative_count)
                        result = "Profit" if positive_count > negative_count else "Loss"
                        print((positive_count/(len(merged_data['price_diff'])))*100)
                        print((negative_count/(len(merged_data['price_diff'])))*100)
                        print((len(merged_data['price_diff'])))
                        print(merged_data['price_diff'])
                        print(f"yhat current: {merged_data['yhat_current']}")
                        print(f"yhat next: {merged_data['yhat_next']}")
                        avgp = round((positive_count if result == "Profit" else negative_count) / len(merged_data) * 100,2)
                        profit_loss_msg = f"You can grow {crop.capitalize()} and expect a {result} of {avgp}% 🪴.<br><br>"

                    response_text += profit_loss_msg

                else:
                    response_text += f"{crop.capitalize()}: but no market data found for prediction.<br><br>"

                response_text += "What do you think about these recommendations?"
                session['feedback_stage'] = True
                return jsonify({'reply': response_text})

            except Exception as e:
                print("Prediction error:", e)
                return jsonify({'reply': 'An error occurred during prediction. Please try again.'})
def handle_soil_conditions(user_message):
    try:
        crop_list = ["bajra", "barley", "turmeric", "tur", "sugarcane", "soybeans",
                    "ragi", "potato", "onion", "maize", "ladyfinger", "jute",
                    "jowar", "green gram", "cotton", "coffee", "chickpea",
                    "cabbage", "wheat", "rice"]

        # Convert message to lowercase and split
        message_parts = user_message.lower().split(' in ')
        crop_part = message_parts[0].strip()

        # Find matching crop (handling partial matches)
        crop_found = None
        for crop in crop_list:
            if crop in crop_part:  # Checks if crop name appears anywhere in the first part
                crop_found = crop
                break

        if not crop_found:
            return jsonify({
                'reply': "Please specify a valid crop. Examples: 'rice in trichy', 'wheat in delhi'",
                'status': 'need_crop'
            })

        # Check if location was provided
        if len(message_parts) < 2:
            return jsonify({
                'reply': f"Please provide a location for {crop_found} (e.g. '{crop_found} in trichy')",
                'status': 'need_location'
            })

        location = message_parts[1].strip()
        temperature, humidity, rainfall = fetch_weather_data(location)
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoder.fit(np.array(crop_list).reshape(-1, 1))
        encoded_crop = encoder.transform([[crop_found]])[0]
        full_features = np.concatenate([encoded_crop, [temperature, humidity, rainfall]])
        input_df = pd.DataFrame([full_features])
        prediction = soil_model.predict(input_df)

        response = {
            'weather': {
                'temperature': temperature,
                'humidity': humidity,
                'rainfall': rainfall
            },
            'soil_conditions': {
                'pH': round(float(prediction[0][0]), 2),
                'N': round(float(prediction[0][1]), 2),
                'P': round(float(prediction[0][2]), 2),
                'K': round(float(prediction[0][3]), 2)
            }
        }

        reply_message = (
            f"Ideal soil conditions for {crop_found}:<br>"
            f"- pH: {response['soil_conditions']['pH']}<br>"
            f"- Nitrogen (N): {response['soil_conditions']['N']}<br>"
            f"- Phosphorus (P): {response['soil_conditions']['P']}<br>"
            f"- Potassium (K): {response['soil_conditions']['K']}<br>"
        )

        return jsonify({
            'reply': reply_message,
            'data': response,
            'status': 'success'
        })

    except Exception as e:
        return jsonify({
            'reply': f"Sorry, I couldn't process that request. Error: {str(e)}",
            'status': 'error'
        })

In [ ]:
!ngrok authtoken 2jB3oS2xptgkYHLMldb5uNwjHEF_3k3Q4UuBthT1xD6nyitXA
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f" * Running on {public_url}")
    app.run()

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Running on NgrokTunnel: "https://5227-34-69-160-14.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=09-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=10-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=11-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed

INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 06:33:35] "GET / HTTP/1.1" 200 -


[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=13-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh


...

[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=25-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=26-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=27-10-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed

DEBUG:cmdstanpy:input tempfile: /tmp/tmpwqsn7cjt/0p4yktkt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwqsn7cjt/ge_vn4tm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=7473', 'data', 'file=/tmp/tmpwqsn7cjt/0p4yktkt.json', 'init=/tmp/tmpwqsn7cjt/ge_vn4tm.json', 'output', 'file=/tmp/tmpwqsn7cjt/prophet_model7n67m_ej/prophet_model-20250427063445.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
06:34:45 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=27-04-2025&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=25-11-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed6-6f13308a1d24?api-key=579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b&format=json&limit=500&filters%5BArrival_Date%5D=26-11-2024&filters%5BCommodity%5D=Rice&filters%5BDistrict%5D=Ghaziabad&filters%5BState%5D=Uttar+Pradesh
[DEBUG] Request URL: https://api.data.gov.in/resource/35985678-0d79-46b4-9ed

06:34:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 06:34:51] "POST /chatbot HTTP/1.1" 200 -


41.66666666666667
58.333333333333336
24
0    -77295.465643
1    -40104.413926
2    -25010.187698
3     38659.961121
4     46188.144998
5    -46326.869350
6    -85403.325299
7    -22630.784477
8     13471.617540
9     25135.617645
10    86252.126307
11    89363.715581
12   -13763.039337
13   -62156.359076
14    -1790.331480
15    29803.746920
16    35932.853891
17    94725.899964
18    89725.887275
19   -29139.282823
20   -93115.562282
21   -45330.691835
22   -38808.209178
23   -84991.259491
Name: price_diff, dtype: float64
yhat current: 0     3246.939538
1     3213.170178
2     3266.501166
3     3184.498632
4     3272.366928
5     3147.516301
6     3002.386102
7     3597.971664
8     3277.185315
9     3181.185180
10    3116.876380
11    3175.107310
12    3370.105848
13    3179.460082
14    3310.676639
15    3188.481392
16    3249.356269
17    3245.314582
18    3233.847518
19    3282.762739
20    3215.906025
21    3268.320556
22    3234.735962
23    3250.637012
Name: yhat_current, dtype